**Задача**

Необходимо написать скрипт AppData, который будет принимать на вход **ID** приложения из iTunes и заносить результаты в **sqlite3 базу данных apps.db** в таблицу **search (app_id, word, position, date)**; в каждой строке должно быть написано написано **id приложения**, запрашиваемое **слово из названия**, **позиция приложения id в выдаче iTunes** по этому слову и **дата запуска скрипта**. 


ложения, которое мы хотим исследовать (например, 860011430). У этого приложения есть название из N слов (в данном случае "Ghost Lens AR Fun Video Editor").


In [4]:
import requests as r
import sqlite3
from datetime import date
import re

In [20]:
def AppData (id):
    url = f'https://itunes.apple.com/lookup?id={id}' #добавляем id в запрос
    response = r.get(url)
    if response.ok:  #проверка на отсутствие ошибки
        result=response.json()
    else:
        print ("Execution failed (search by ID)") #добавила конкретизацию шага, на котором произошла ошибка
    if 'results' in result and len(result['results']) != 0: #проверяем, что получили не пустой results
        app_name=result['results'][0]['trackName'] #получаем название приложения
    else:
        print(f'Error, data on app ID {id} not found') #проверяем, что данные получены, если нет - сообщение об ошибке
    words = re.findall(r'\w+', app_name)  #находим слова в названии приложения, знаки препинания не записываем
    current_date = date.today().isoformat() #получаем дату и приводим к формату "yyyy-mm-dd"
    con = sqlite3.connect("apps.db") #Устанавливаем соединение с базой
    cur = con.cursor()
    #Создадим таблицу, если ее нет. Если мы уверены, что таблица существует, этот шаг можно убрать
    cur.execute('''
    CREATE TABLE IF NOT EXISTS search (
        app_id INTEGER,
        word TEXT,
        position INTEGER,
        date TEXT
    )
    ''')
    for word in words: #итерируемся по каждому слову из списка слов
        prep_and_articles=('in','the','a','and','by','as','at','with','of','for','on','into','to')
        #кортеж из слов, по которым поиск не информативен, на случай, если они будут в названии
        if word.lower() not in prep_and_articles: #приводим к нижнему регистру, чтобы соотнести со словом в кортеже
            word_url = f'https://itunes.apple.com/search?term={word}&entity=software&limit=200'#по умолчанию 50 страниц, мы доводим до максимума - 200
            #Допустим, что ищем по приложениям, если нужно искать по всему контенту, entity можно убрать
            response_search = r.get(word_url)
            if response_search.ok:  #проверка на отсутствие ошибки
                result_search=response_search.json()
            else:
                print ("Execution failed (search by words)") #добавила конкретизацию шага, на котором произошла ошибка
            request=word #создадим переменную со словом из запроса
            position = None #Инициализируем переменную
            if 'results' in result_search and len(result_search['results']) != 0: #поиск позиции осуществляем, если результат не пустой
                for index, element in enumerate (result_search['results']): #итерируемся по results
                    if element['trackId'] == id: #Ищем соответствие по ID приложения
                        position = index + 1 #т.к. нумерация начинается с 0
                        break
            else:
                print("Empty search result")
            obtained_data=[id, word, position, current_date]
            cur.execute("INSERT INTO search VALUES(?,?,?,?)", obtained_data) #добавляем значения в таблицу
            print(f'Inserted: {obtained_data}')
            if position is None:
                print(f'App not found for word: {word}') #Если позиция не найдена, выводим предупреждение, но None в базу все равно записываем
    con.commit()
    con.close()